In [142]:
import pandas as pd
import requests
import json
from pattern import web
pd.options.display.max_rows = 200

In [163]:
# load emails from the google groups export
emails = pd.DataFrame().from_csv("5sfc.csv").reset_index()
emails["nickname"] = emails.Nickname.map(lambda x: x.lower() if type(x) is str else x)
emails["email_handle"] = emails["Email address"].map(lambda x: x[:x.find("@")].lower() if type(x) is str else x)

In [159]:
master = requests.get("http://www.forbes.com/30-under-30-2016/").text

start_str = "fbs_settings.promoted_content = "
end_str = ";\n\t\t} catch(err)"

start = master.find(start_str) + len(start_str)
end = master[start:].find(end_str)

# retrieve the json-ified content
master_content = json.loads(doc[start:][:end])

# prepare the list of names
f30 = []
f30_category = []

# find the tile blocks
for key in master_content:
    try:
        if master_content[key]['contentPositions'][0]['description'].find(" Tile") != -1:
            subsection = master_content[key]['contentPositions']

            # go through each category
            for c1 in subsection:
                dom = web.DOM(c1['description'])

                categories = dom("a.category")
                for c2 in categories:
                    category_link = c2.href
                    category_name = category_link.split("/")[-1]
                    cat = requests.get(category_link).text

                    print "Browsing category {0}".format(category_name)

                    # get the content position indices
                    start = cat.find(start_str) + len(start_str)
                    end = cat[start:].find(end_str)

                    # retrieve the json-ified content
                    content = json.loads(cat[start:][:end])

                    # capture the keys that contain people data
                    content_blocks = []

                    for i in content:
                        try:
                            # look for contents that include the property 'age'. These are our 30 Under 30 tiles
                            content[i]['contentPositions'][0]['age']
                            content_blocks.append(i)
                        except KeyError:
                            try:
                                
                                # get the featured person
                                content[i]['contentPositions'][0]['byline']
                                f30.append(content[i]['contentPositions'][0]['byline'])
                                f30_category.append(category_name)
                            except KeyError:
                                continue

                    for i in content_blocks:
                        for j in content[i]['contentPositions']:
                            try:
                                # find recipients in teams
                                for k in j['members']:
                                    f30.append(k['name'])
                                    f30_category.append(category_name)
                            except KeyError:
                                # find solo recipients
                                f30.append(j['title'])
                                f30_category.append(category_name)
    except KeyError:
        continue


Browsing category art-style
Browsing category finance
Browsing category consumer-tech
Browsing category sports
Browsing category hollywood-entertainment
Browsing category retail-ecommerce
Browsing category education
Browsing category music
Browsing category games
Browsing category venture-capital
Browsing category media
Browsing category manufacturing-industry
Browsing category enterprise-tech
Browsing category food-drink
Browsing category social-entrepreneurs
Browsing category law-policy
Browsing category science
Browsing category marketing-advertising
Browsing category energy
Browsing category healthcare


In [180]:
df = pd.DataFrame({"full_name" : f30, "category" : f30_category})#.to_csv("forbes_30under30_list.csv", encoding='utf-8')

In [161]:
df["firstn"] = df.full_name.apply(lambda x: x.split(" ")[0])
df["lastn"] = df.full_name.apply(lambda x: x.split(" ")[-1])
last_names = list(df.lastn.map(lambda x: x.lower()).values)
full_names = list(df.full_name.values)

In [162]:
df.head()

,category,full_name,firstn,lastn
0,art-style,Ashley Graham,Ashley,Graham
1,art-style,Madison Maxey,Madison,Maxey
2,art-style,Athena Papadopoulos,Athena,Papadopoulos
3,art-style,Ryder Ripps,Ryder,Ripps
4,art-style,Noa Santos,Noa,Santos


In [167]:
# search by last name (horribly inefficient)
# for each last name in the Forbes list, see if it is found in either the subscriber's email or nickname
email_search = emails.apply(lambda x: [i for i in range(0, len(last_names)) if x.email_handle.find(last_names[i]) != -1 or (x.nickname.find(last_names[i]) != -1 if type(x.Nickname) is str else False) ], axis=1)

# filter for found results
matches = email_search.map(len) > 0
matched = emails[matches]
email_search = email_search[matches]

In [178]:
# full names of those who matched by last name
emails["possible_matches"] = email_search.map(lambda x: [full_names[i] for i in x])

# if the Nickname is filled out, see if there is an exact last name match. if so, see if the first letter of the first name also matches. 
emails["possible_matches"] = emails[matches].apply(lambda x:  [ i for i in x.possible_matches if (x.nickname.split(" ")[-1] == i.split(" ")[-1].lower()) and (x.nickname[0] == i[0].lower()) ] if type(x.Nickname) is str and len(x.Nickname.split(" ")) > 1 else x.possible_matches, axis=1)

# see if there's any easy exact matches. Only if the nickname column was filled out
emails["exact_matches"] = emails.apply(lambda x: [i for i in x.possible_matches if type(x.Nickname) is str and x.Nickname.strip() == i] if type(x.possible_matches) is list else None, axis=1)

# show exact matches
emails[matches][emails[matches].exact_matches.map(len) > 0][["Nickname", "exact_matches"]]


,Nickname,exact_matches
48,Brian Powers,"[Brian Powers, Brian Powers]"
63,Christopher Gray,[Christopher Gray]
101,Liz Wessel,[Liz Wessel]
155,Jesse Beyroutey,[Jesse Beyroutey]
323,Spencer Penn,[Spencer Penn]


In [179]:
# export possible matches
emails[matches][emails[matches].possible_matches.map(len) > 0][["Email address","Nickname", "possible_matches","exact_matches"]].to_csv("F30U30_5SFC.csv")